# Segmentation

<br />

I will do segmentation for a medical dataset, "TGS Salt Identification Challenge" on Kaggle.


https://www.kaggle.com/c/tgs-salt-identification-challenge

<br />

In this task, I am going to use U-Net as segmentation method.

<br />

"U-Net: Convolutional Networks for Biomedical Image Segmentation"


Olaf Ronneberger, Philipp Fischer, and Thomas Brox


https://arxiv.org/pdf/1505.04597.pdf

## [Task 1] Fit and Predict

<br />

I fit and predict the dataset by using the following implementation of Keras.


https://github.com/zhixuhao/unet

    -> sprint20-colab.ipynb

## [Task 2] Code Reading

<br />

model.py


https://github.com/zhixuhao/unet/blob/master/model.py

In [ ]:
# Definition of U-Net

def unet(pretrained_weights = None,input_size = (256,256,1)):
    # Down sampling (Encoder)
    inputs = Input(input_size)
    
    # Lv.4 (conv 3x3, ReLU)
    conv1 = Conv2D(filters = 64, kernel_size = 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    
    # Pooling (max pool 2x2)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Lv.3 (conv 3x3, ReLU)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    
    # Pooling (max pool 2x2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Lv.2 (conv 3x3, ReLU)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    
    # Pooling (max pool 2x2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # Lv.1
    # Conv 3x3, ReLU
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    # Dropout
    drop4 = Dropout(0.5)(conv4)
    
    # Pooling (max pool 2x2)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
    # Lv.0
    # Conv 3x3, ReLU
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    # Dropout
    drop5 = Dropout(0.5)(conv5)
    
    
    # Up sampling (Decoder)
    # Upsampling (up-conv 2x2)
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    
    # Lv.1
    # Copy and crop
    merge6 = concatenate([drop4,up6], axis = 3)
    # Conv 3x3, ReLU
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    
    # Upsampling (up-conv 2x2)
    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    
    # Lv.2
    # Copy and crop
    merge7 = concatenate([conv3,up7], axis = 3)
    # Conv 3x3, ReLU
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    
    # Upsampling (up-conv 2x2)
    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    
    # Lv.3
    # Copy and crop
    merge8 = concatenate([conv2,up8], axis = 3)
    # Conv 3x3, ReLU
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    
    # Upsampling (up-conv 2x2)
    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    
    # Lv.4
    # Copy and crop
    merge9 = concatenate([conv1,up9], axis = 3)
    # Conv 3x3, ReLU
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    # Conv 1x1, Sigmoid
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    
    # Initialize
    model = Model(input = inputs, output = conv10)
    
    # Compile
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    # Check the model
    #model.summary()
    
    # Load weights
    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model